In [1]:
import numpy as np
import polars as pl
import time

In [2]:
blocks = pl.read_parquet('../out/train/building_blocks.parquet')
blocks.head()

index,smile,ecfp,ecfp_pca,onehot_pca
i64,str,list[i64],list[f64],list[f64]
0,"""Br.Br.NCC1CCCN…","[0, 0, … 0]","[-0.623631, 1.179455, … -0.077481]","[-4.098411, 0.498678, … -0.011187]"
1,"""Br.NCc1cccc(Br…","[0, 0, … 0]","[-1.059422, 0.214484, … 0.01271]","[-3.521219, -3.102144, … 0.240913]"
2,"""C#CCOc1ccc(CN)…","[0, 0, … 0]","[-0.7829, 0.581478, … -0.098236]","[-1.921474, 0.660365, … 0.950661]"
3,"""C#CCOc1cccc(CN…","[0, 0, … 0]","[-0.613241, 0.365505, … -0.046728]","[-1.921474, 0.660365, … 0.950661]"
4,"""C#CC[C@@H](CC(…","[0, 1, … 0]","[3.472075, 0.095289, … -0.012011]","[6.583925, 4.480118, … 0.274481]"


In [3]:
train = pl.read_parquet('../out/train/train/base/base-BRD4-001.parquet')
train.head()

id,protein_name,binds,buildingblock1_index,buildingblock2_index,buildingblock3_index
i64,str,i64,i64,i64,i64
4358142,"""BRD4""",0,8,3,0
4358145,"""BRD4""",0,8,3,1
4358151,"""BRD4""",0,8,3,3
4358154,"""BRD4""",0,8,3,10
4358157,"""BRD4""",0,8,3,11


In [4]:
def features(dt, blocks):
    iblocks_ecfp_pca = [np.vstack(blocks['ecfp_pca'][dt[x]]) for x in ['buildingblock1_index', 'buildingblock2_index', 'buildingblock3_index']]
    iblocks_ecfp_pca = np.concatenate(iblocks_ecfp_pca, axis = 1)
    return(iblocks_ecfp_pca)

start_time = time.time()
x = features(train, blocks)
print(f"Elapsed time: {(time.time() - start_time)/60:.2f} minutes")
print(x[0:5])

# 1.41 mins

Elapsed time: 1.41 minutes
[[ 3.52322333  0.13099574  1.66057898 ... -0.04508049  0.1078245
  -0.07748132]
 [ 3.52322333  0.13099574  1.66057898 ...  0.05860765 -0.0295106
   0.01271006]
 [ 3.52322333  0.13099574  1.66057898 ... -0.05801682 -0.00543795
  -0.04672798]
 [ 3.52322333  0.13099574  1.66057898 ...  0.01891126  0.03817647
   0.07556934]
 [ 3.52322333  0.13099574  1.66057898 ... -0.01334688  0.04931808
  -0.00601576]]


In [7]:
def features(dt, blocks):
    iblocks_ecfp_pca = [[list(ix) for ix in blocks['ecfp_pca'][dt[x]]] for x in ['buildingblock1_index', 'buildingblock2_index', 'buildingblock3_index']]
    iblocks_ecfp_pca = np.concatenate(iblocks_ecfp_pca, axis = 1)
    return(iblocks_ecfp_pca)

start_time = time.time()
x = features(train, blocks)
print(f"Elapsed time: {(time.time() - start_time)/60:.2f} minutes")
print(x[0:5])

#  mins

MemoryError: Unable to allocate 2.09 GiB for an array with shape (651029, 430) and data type float64

In [5]:
# def features(dt, blocks):
    
#     blocks_ecfp_pca = []
#     for i in ['buildingblock1_index', 'buildingblock2_index', 'buildingblock3_index']:
#         iblocks_ecfp_pca = blocks['ecfp_pca'][dt[i]].to_numpy()
#         iblocks_ecfp_pca = np.array([list(x) for x in iblocks_ecfp_pca]).astype('float')
#         blocks_ecfp_pca.append(iblocks_ecfp_pca)
#         del iblocks_ecfp_pca, i
        
#     blocks_ecfp_pca = np.concatenate(blocks_ecfp_pca, axis = 1)
#     return(blocks_ecfp_pca)

# start_time = time.time()
# x = features(train, blocks)
# print(f"Elapsed time: {(time.time() - start_time)/60:.2f} minutes")
# print(x[0:5])

Elapsed time: 5.05 minutes
[[ 3.52322333  0.13099574  1.66057898 ... -0.04508049  0.1078245
  -0.07748132]
 [ 3.52322333  0.13099574  1.66057898 ...  0.05860765 -0.0295106
   0.01271006]
 [ 3.52322333  0.13099574  1.66057898 ... -0.05801682 -0.00543795
  -0.04672798]
 [ 3.52322333  0.13099574  1.66057898 ...  0.01891126  0.03817647
   0.07556934]
 [ 3.52322333  0.13099574  1.66057898 ... -0.01334688  0.04931808
  -0.00601576]]


In [6]:
def features(dt, blocks):

    blocks = blocks.select(['index', 'ecfp_pca'])
    dt = dt.select(['id', 'buildingblock1_index', 'buildingblock2_index', 'buildingblock3_index'])#.sample(10)

    dt = dt.join(blocks, left_on = 'buildingblock1_index', right_on = 'index', how = 'inner').drop('buildingblock1_index').rename({'ecfp_pca': 'ecfp_pca1'})
    dt = dt.join(blocks, left_on = 'buildingblock2_index', right_on = 'index', how = 'inner').drop('buildingblock2_index').rename({'ecfp_pca': 'ecfp_pca2'})
    dt = dt.join(blocks, left_on = 'buildingblock3_index', right_on = 'index', how = 'inner').drop('buildingblock3_index').rename({'ecfp_pca': 'ecfp_pca3'})

    dt = dt.with_columns(
        pl.col('ecfp_pca1').list.concat(pl.col('ecfp_pca2').list.concat(pl.col('ecfp_pca3'))).alias('blocks_ecfp_pca')
    ).select(['id', 'blocks_ecfp_pca'])

    return(np.vstack(dt['blocks_ecfp_pca']))

start_time = time.time()
x = features(train, blocks)
print(f"Elapsed time: {(time.time() - start_time)/60:.2f} minutes")
print(x[0:5])

Elapsed time: 1.85 minutes
[[ 3.52322333  0.13099574  1.66057898 ... -0.04508049  0.1078245
  -0.07748132]
 [ 3.52322333  0.13099574  1.66057898 ...  0.05860765 -0.0295106
   0.01271006]
 [ 3.52322333  0.13099574  1.66057898 ... -0.05801682 -0.00543795
  -0.04672798]
 [ 3.52322333  0.13099574  1.66057898 ...  0.01891126  0.03817647
   0.07556934]
 [ 3.52322333  0.13099574  1.66057898 ... -0.01334688  0.04931808
  -0.00601576]]
